### Lab IV - Performance Tunning

In [ ]:
import cProfile
import re
import numpy as np
import scipy.signal
import matplotlib.pyplot as plt
%load_ext line_profiler

### 1.Taylor series expansion

$$e^x = \sum^\infty_{n=0} 1 + x + \frac{x^2}{2!} + \frac{x^3}{3!} + ...  $$


$$\sin(x) = x - \frac{x^3}{3!} + \frac{x^5}{5!} + ... $$

In [ ]:
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

def taylor_exp(n):
    res = []
    for i in range(n):
        res.append(1 / factorial(i))
    return res


In [ ]:
%lprun -f taylor_exp taylor_exp(1000)


In [ ]:
def factorial_upto(n):
    res = [1]
    
    for i in range(1, n + 1):
        res.append(res[-1] * i)
    return res

def better_taylor_exp(n):
    
    factorials = factorial_upto(n)
    
    res = list(map(lambda x: 1/x, factorials))
    return res

In [ ]:
%lprun -f better_taylor_exp better_taylor_exp(1000)


### 2. Heat Equation
Lets further analyze the problem of solving the _heat equation_:
$$
\frac{\partial^2 f}{\partial x^2}+\frac{\partial^2 f}{\partial y^2}=0
$$
using central difference in a square $[0,1]\times[0,1]$ regular grid:
$$
\frac{\partial^2 f}{\partial x^2}\approx\frac{f_{i+1,j}-f_{i,j}+f_{i-1,j} - f_{i,j}}{\Delta x^2} = \frac{f_{i+1,j}+f_{i-1,j} - 2f_{i,j}}{\Delta x^2}
$$

$$
\frac{\partial^2 f}{\partial y^2}\approx \frac{f_{i+1,j}-f_{i,j}+f_{i-1,j} - f_{i,j}}{\Delta y^2} = \frac{f_{i,j+1}+f_{i,j-1} - 2f_{i,j}}{\Delta y^2}
$$

We will assume $\Delta x = \Delta y$, so it follows:
$$
f_{i,j} = \frac{f_{i+1,j}+f_{i-1,j} + f_{i,j+1} + f_{i,j-1}}{4} \,.
$$

We set up initial value at boundary.

Two versions of the code presented in class:

In [ ]:
from itertools import product

def build_grid(n):
    '''defines the grid'''
    G = np.zeros((n,n))
    return(G)

def set_boundary_grid(G):
    '''set the boundary values'''
    n = G.shape[0]
    G[:,0] = np.ones((1,n))
    G[0,:] = np.ones((1,n)) 
    G[-1,:] = np.ones((1,n))
    G[:,-1] = np.ones((1,n))
    
def calc_error(G1,G2):
    error = np.square(np.subtract(G1, G2)).mean()
    return(error)

def copy_array(G1,G2):
    G2[:] = G1[:]
    

In [ ]:
# we are passing the function that updates G as a parameter
def solve_heat_equation(n,update):
    G = build_grid(n)
    set_boundary_grid(G)
    G_prev = build_grid(n)
    copy_array(G, G_prev)
    error = float('inf')
    while error > 1e-7:
        update(G)
        error = calc_error(G,G_prev)
        copy_array(G, G_prev)    
        
    return(G)

In [ ]:
def itera(G):
    n = G.shape[0]
    for i in range(1,n-1):
        for j in range(1,n-1):
            G[i,j]=(G[i+1,j]+G[i-1,j]+G[i,j+1]+G[i,j-1])/4
            
 

#### Convolution 

A mathematical operation between two functions f,h that produces a third function.

$$y = x * h$$
$$y = \sum^{a}_{s = -a} \sum^{b}_{t = -b} h(s,t) f(x-s, y-t)  $$

In [ ]:
def itera_conv(G):
    n = G.shape[0]
    f = np.array([[0,1/4,0], [1/4,0,1/4], [0,1/4,0]])
    G[1:n-1,1:n-1] = scipy.signal.convolve(G, f, 'valid')
    

In [ ]:
sol1 = solve_heat_equation(70,itera)
sol2 = solve_heat_equation(70,itera_conv)


In [ ]:
plt.figure(figsize = (10,5))
plt.subplot(1,2,1)
plt.title('Original')
img=plt.imshow(sol1)
img.set_cmap('rainbow')
plt.axis('off')

plt.subplot(1,2,2)
plt.title('Convolution')
img=plt.imshow(sol2)
img.set_cmap('rainbow')
plt.axis('off')
plt.show()

In [ ]:
def itera_slicing(G):
    n = G.shape[0]

    Gl = G[1:-1,:-2]  #Left neighbors
    Gr = G[1:-1,2:]   #Right neighbors
    Gt = G[2:,1:-1]   #top
    Gb = G[:-2,1:-1]  #bottom
    
    G[1:n-1,1:n-1] = (Gl + Gr + Gt + Gb)/4
    
sol3 = solve_heat_equation(70,itera_slicing)

In [ ]:
plt.figure(figsize = (10,5))
plt.subplot(1,2,1)
plt.title('Original')
img=plt.imshow(sol1)
img.set_cmap('rainbow')
plt.axis('off')

plt.subplot(1,2,2)
plt.title('Slicing')
img=plt.imshow(sol3)
img.set_cmap('rainbow')
plt.axis('off')
plt.show()

Profile the code above to analyze the computational time of each line of code

In [ ]:
# Solution
%lprun -f solve_heat_equation solve_heat_equation(70,itera)

In [ ]:
%lprun -f solve_heat_equation solve_heat_equation(70,itera_conv)

In [ ]:
%lprun -f solve_heat_equation solve_heat_equation(70,itera_slicing)

Based on your analysis, how can you further improve the computational performance of the code?

In [ ]:
# Improvement 1: remove the calls to copy_array

def solve_heat_equation2(n,update):
    G = build_grid(n)
    set_boundary_grid(G)
    G_prev = build_grid(n)
    copy_array(G, G_prev)
    error = 1e10
    while error > 1e-7:
        update(G)
        error = calc_error(G,G_prev)
        G_prev[:] = G[:]              # Not calling the copy_array function anymore
        
    return(G)

In [ ]:
%lprun -f solve_heat_equation2 solve_heat_equation2(70,itera_slicing)

In [ ]:
# Improvement 2: removing the call to calc_error

def solve_heat_equation3(n,update):
    G = build_grid(n)
    set_boundary_grid(G)
    G_prev = build_grid(n)
    copy_array(G, G_prev)
    error = 1e10
    square = np.square
    subtract = np.subtract
    
    while error > 1e-7:
        update(G)
        error = square(subtract(G, G_prev)).mean()  # Not calling the calc_error function anymore
        G_prev[:] = G[:]             
        
    return(G)

In [ ]:
%lprun -f solve_heat_equation3 solve_heat_equation3(70,itera_slicing)

In [ ]:
%timeit solve_heat_equation3(70, itera_slicing)
%timeit solve_heat_equation2(70, itera_slicing)
